In [5]:
!pip install langchain langchain-community tiktoken pypdf rapidocr-onnxruntime
!pip install -U weaviate-client
!pip install -Uqq langchain-weaviate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.1/519.1 kB 15.7 MB/s eta 0:00:00


In [6]:
WEAVIATE_API_KEY = ""
WEAVIATE_CLUSTER = "https:"

In [13]:
from langchain.vectorstores import Weaviate
import weaviate
from weaviate.classes.init import Auth

WEAVIATE_URL = WEAVIATE_CLUSTER
WEAVIATE_API_KEY = WEAVIATE_API_KEY



In [ ]:
dir(weaviate)

https://weaviate.io/developers/weaviate/connections/connect-cloud

In [14]:
# Connect to Weaviate Cloud
client = weaviate.connect_to_weaviate_cloud(
    cluster_url= WEAVIATE_URL,
    auth_credentials=Auth.api_key(WEAVIATE_API_KEY),
)

print(client.is_ready())

True


In [15]:
import locale
locale.getpreferredencoding = lambda : "UTF-8"

In [16]:
!pip install sentence-transformers

In [17]:
from langchain.embeddings import HuggingFaceEmbeddings
embedding_model_name = "sentence-transformers/all-mpnet-base-v2"

embeddings = HuggingFaceEmbeddings(
    model_name = embedding_model_name
)

<ipython-input-17-2fb581132ad5>:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

https://python.langchain.com/docs/modules/data_connection/document_loaders/pdf/

In [18]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("/content/temp.pdf",extract_images=True)
pages = loader.load()

In [19]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=20)
docs = text_splitter.split_documents(pages)

docs

[Document(metadata={'source': '/content/temp.pdf', 'page': 0}, page_content='A 76-Year-Old Woman  \nwith Stomach Cancer\nJOHN MULLEN, MD\nMassachusetts General Hospital\nPRESENTATION OF CASE\nA 76-year-old woman was seen in our \nmultidisciplinary gastrointestinal oncology clinic \nfor further management of an adenocarcinoma of \nthe body of the stomach. \nThe patient presented to her primary care \nphysician with a four-week history of intermittent \nchest pressure radiating to her back. Extensive \nevaluation for a cardiac source was unremarkable, \nincluding a normal stress test and abdominal \nultrasound. She then developed epigastric \nabdominal pain, and given that she carried \na history of prior H. pylori infection in 2003, \nthere was concern that she might have a peptic \nulcer. An upper endoscopy was obtained and \nshowed a large ulcerated tumor in the body of \nthe stomach (Figure 1). Pathological examination \nof biopsy specimens of the stomach ulcer \nshowed moderately to

In [21]:

from langchain_weaviate.vectorstores import WeaviateVectorStore

In [ ]:
dir(Weaviate)

In [ ]:
# Connect to Weaviate Cloud
client = weaviate.connect_to_weaviate_cloud(
    cluster_url= WEAVIATE_URL,
    auth_credentials=Auth.api_key(WEAVIATE_API_KEY),
)

print(client.is_ready())

In [ ]:
"""client = weaviate.Client(
    url=WEAVIATE_URL, auth_client_secret=weaviate.AuthApiKey(WEAVIATE_API_KEY)
)
"""

In [32]:
db = WeaviateVectorStore.from_documents(docs, embeddings, client=client, index_name="LangChain")


In [33]:

collection = client.collections.get("LangChain")

In [35]:
collection

In [36]:
response = collection.aggregate.over_all(total_count=True)
print(response)

AggregateReturn(properties={}, total_count=11)


In [37]:
response = collection.aggregate.over_all(group_by="source")
for group in response.groups:
    print(group.grouped_by.value, group.total_count)

/content/temp.pdf 11


In [38]:
object = collection.query.fetch_objects(limit=1).objects[0]

In [39]:
object.properties.keys()

dict_keys(['text', 'page', 'source'])

In [40]:
print(object.properties.get("source"))
print(object.properties.get("page"))
print(object.properties.get("text"))

/content/temp.pdf
2.0
manual. 7th ed. New York: Springer-Verlag;2009:117-126.
2. Macdonald JS, Smalley SR, Benedetti J, et al. 
Chemoradiotherapy after surgery compared with surgery alone 
for adenocarcinoma of the stomach or gastroesophageal 
junction. N Engl J Med. 2001;345:725-730.
3. Cunningham D, Allum WH, Stenning SP , et al. Perioperative 
chemotherapy versus surgery alone for resectable 
gastroesophageal cancer . N Engl J Med. 2006;355:11-20.
4. Ychou M, Boige V , Pignon JP , et al. Perioperative chemotherapy 
compared with surgery alone for resectable gastroesophageal 
adenocarcinoma: an FNCLCC and FFCD multicenter phase III 
trial. J Clin Oncol. 2011;29:1715-1721.
5. Schmidt B, Chang KK, Maduekwe UN, et al. D2 
lymphadenectomy with surgical ex vivo dissection into node 
stations for gastric adenocarcinoma can be performed safely in 
Western patients and ensures optimal staging. Ann Surg Oncol 
2013;20:2991-2999.
6. Mullen JT. Subtotal gastrectomy for gastric cancer . In:  Fis

In [44]:
print(db.similarity_search("what is Stomach Cancer?", k=3)[0].page_content)


never smoked and did not drink alcohol. There 
was no family history of stomach cancer, but her 
older brother died of colon cancer metastatic to 
the liver and lung at the age of 90.
On examination, the weight was 182 pounds, the 
blood pressure 182/90 mm Hg, the pulse 90 beats 
per minute and the temperature 97.2 degrees 
Fahrenheit. There was mild epigastric tenderness 
without a palpable mass nor an enlarged liver. 
Results of a complete blood count, plasma levels  
Page 1 of 3
Gastroesophageal Surgery Case
FIGURE 1. Image from upper endoscopic examination 
demonstrating a large ulcerated tumor in the body  
of the stomach.


In [45]:
from langchain.prompts import ChatPromptTemplate

template="""You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use ten sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""

In [46]:
prompt=ChatPromptTemplate.from_template(template)

In [60]:

from langchain import HuggingFaceHub
huggingfacehub_api_token = "hf_"

In [61]:
model = HuggingFaceHub(
    huggingfacehub_api_token=huggingfacehub_api_token,
    repo_id="TheBloke/Mistral-7B-Instruct-v0.1-GGUF",
    model_kwargs={"temperature":1, "max_length":180}
)

In [62]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.base.Runnable.html

In [63]:
output_parser = StrOutputParser()

In [64]:
retriever = db.as_retriever()

In [65]:
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | output_parser

)

In [ ]:
rag_chain.invoke("what is Stomach Cancer?")